In [1]:
import numpy as np
import pandas as pd
import os


path = './data/'
output = './output/'



In [2]:
df = pd.read_csv(path+'RegularSeasonDetailedResults.csv')
print(len(df))
print(list(df))

82041
['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']


In [3]:
print(df.Season.unique())

[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018]


In [4]:
df = df.loc[df.Season>=2010]
print(len(df))

47967


In [5]:


new_cols = []
for column in list(df):
    if column == 'WLoc':
        new_cols.append('WLoc')
    else:
        column = column.replace("W","")
        column = column.replace("L","Opp")
        new_cols.append(column)
    
print(new_cols)


['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'WLoc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']


In [6]:
df.columns=new_cols


In [7]:
print(df.WLoc.unique())

['H' 'N' 'A']


In [8]:
opp_df = df.copy()
opp_cols = list(opp_df)

static_cols = ['Season','DayNum','WLoc','NumOT']
new_oc = []

for col in opp_cols:
    if col in static_cols:
        new_oc.append(col)
    elif "Opp" in col:
        new_oc.append(col.replace("Opp",""))
    else:
        new_oc.append('Opp'+col)

print(new_oc)

opp_df.columns=new_oc



['Season', 'DayNum', 'OppTeamID', 'OppScore', 'TeamID', 'Score', 'WLoc', 'NumOT', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']


In [9]:
# adjust location
reverse = {
    'H':'A',
    'A':'H',
    'N':'N'
}

opp_df['WLoc'] = opp_df['WLoc'].replace(reverse)
print(opp_df.WLoc.head())

34074    A
34075    A
34076    A
34077    A
34078    A
Name: WLoc, dtype: object


In [10]:
# rearrange columns to match original
opp_df = opp_df[new_cols]
print(list(opp_df))



['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'WLoc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']


In [11]:
print(len(df))
df = pd.concat([df,opp_df],axis=0)
print(len(df))

df = df.rename(columns={'WLoc':'Loc'})

47967
95934


In [12]:
# possessions calc
# field goals attempted - offensive rebounds + turnovers + (0.4 x free throws attempted)

df['Poss'] = df['FGA']-df['OR']+df['TO']+(0.4*df['FTA'])
df['OppPoss'] = df['OppFGA']-df['OppOR']+df['OppTO']+(0.4*df['OppFTA'])
df['Poss'] = np.round((df['Poss']+df['OppPoss'])/2,1)
df = df.drop(columns=['OppPoss'])

df['Poss'].head()

34074    63.9
34075    82.2
34076    70.9
34077    80.0
34078    69.4
Name: Poss, dtype: float64

Desired YOY stats...
FG%,
FGM3%,
Rebound%,
Assist,Steals,Turnovers,Blocks per Poss?
Pace


In [13]:
# SeasonTeamID
df['SeaTeamID'] = df['Season'].astype(str)+df['TeamID'].astype(str)

# only need a some columns 
desired_cols = ['SeaTeamID','Season','TeamID','FGM','FGA','FGM3','FGA3','OR','DR',
                'OppFGM','OppFGA','OppFGM3','OppFGA3','OppOR','OppDR','Poss']

gb = df[desired_cols]


# first groupby Team/Season

gb = gb.groupby(['SeaTeamID'])['FGM','FGA','FGM3','FGA3','OR','DR','OppFGM','OppFGA','OppFGM3','OppFGA3','OppOR','OppDR','Poss'].mean().reset_index()
gb = gb.rename(columns={'Poss':'Pace'})

gb.head()



,SeaTeamID,FGM,FGA,FGM3,FGA3,OR,DR,OppFGM,OppFGA,OppFGM3,OppFGA3,OppOR,OppDR,Pace
0,20101102,20.000000,45.586207,5.620690,18.103448,6.758621,19.862069,21.448276,47.103448,7.206897,20.827586,8.862069,22.206897,57.662069
1,20101103,25.090909,57.636364,6.606061,19.515152,13.454545,22.696970,22.787879,53.787879,4.969697,15.272727,11.424242,22.242424,66.060606
2,20101104,24.718750,56.062500,5.468750,15.625000,12.062500,23.125000,23.000000,54.187500,5.656250,17.812500,12.000000,22.437500,64.268750
3,20101105,21.173913,57.173913,3.826087,13.565217,13.956522,21.608696,24.826087,56.913043,5.130435,16.347826,15.260870,27.173913,70.213043
4,20101106,21.214286,53.500000,5.928571,19.071429,13.357143,22.321429,22.357143,50.857143,5.000000,14.035714,10.250000,23.607143,66.285714


In [14]:


gb['FG%']  = gb['FGM']/gb['FGA']
gb['FGM3%'] = gb['FGM3']/gb['FGA3']

#allowed 
gb['A_FG%']  = gb['OppFGM']/gb['OppFGA']
gb['A_FGM3%'] = gb['OppFGM3']/gb['OppFGA3']

#defensive rebound rate
gb['DRR'] = gb['DR']/(gb['DR']+gb['OppOR'])

#offensive rebound rate
gb['ORR'] = gb['OR']/(gb['OR']+gb['OppDR'])

gb = gb[['SeaTeamID','FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']]

gb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace
0,20101102,0.438729,0.310476,0.455344,0.346026,0.691477,0.233333,57.662069
1,20101103,0.435331,0.338509,0.423662,0.325397,0.665187,0.376910,66.060606
2,20101104,0.440914,0.350000,0.424452,0.317544,0.658363,0.349638,64.268750
3,20101105,0.370342,0.282051,0.436211,0.313830,0.586085,0.339323,70.213043
4,20101106,0.396529,0.310861,0.439607,0.356234,0.685307,0.361353,66.285714


In [15]:
teams = df[['SeaTeamID','Season','TeamID']]
teams = teams.drop_duplicates()

gb = pd.merge(left=gb,right=teams,how='left',on=['SeaTeamID','SeaTeamID'])


In [16]:
gb['NextSeason'] = gb['Season'] + 1
gb['NextSeaTID'] = gb['NextSeason'].astype(str) + gb['TeamID'].astype(str)
gb = gb.drop(columns=['NextSeason'])
gb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace,Season,TeamID,NextSeaTID
0,20101102,0.438729,0.310476,0.455344,0.346026,0.691477,0.233333,57.662069,2010,1102,20111102
1,20101103,0.435331,0.338509,0.423662,0.325397,0.665187,0.376910,66.060606,2010,1103,20111103
2,20101104,0.440914,0.350000,0.424452,0.317544,0.658363,0.349638,64.268750,2010,1104,20111104
3,20101105,0.370342,0.282051,0.436211,0.313830,0.586085,0.339323,70.213043,2010,1105,20111105
4,20101106,0.396529,0.310861,0.439607,0.356234,0.685307,0.361353,66.285714,2010,1106,20111106


In [17]:
ns = gb.copy()

ns = ns[['SeaTeamID', 'FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']]

new_cols=[]
for col in list(ns):
    new_cols.append('N'+col)
    
ns.columns=new_cols
ns = ns.rename(columns={'NSeaTeamID':'NextSeaTID'})

tgb = pd.merge(left=gb,right=ns,how='left',on=['NextSeaTID','NextSeaTID'])

tgb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace,Season,TeamID,NextSeaTID,NFG%,NFGM3%,NA_FG%,NA_FGM3%,NDRR,NORR,NPace
0,20101102,0.438729,0.310476,0.455344,0.346026,0.691477,0.233333,57.662069,2010,1102,20111102,0.468657,0.376258,0.411141,0.337681,0.656904,0.203980,61.575000
1,20101103,0.435331,0.338509,0.423662,0.325397,0.665187,0.376910,66.060606,2010,1103,20111103,0.429820,0.360215,0.417017,0.318339,0.669106,0.284526,66.502941
2,20101104,0.440914,0.350000,0.424452,0.317544,0.658363,0.349638,64.268750,2010,1104,20111104,0.444941,0.298405,0.383353,0.318267,0.663221,0.372458,64.521875
3,20101105,0.370342,0.282051,0.436211,0.313830,0.586085,0.339323,70.213043,2010,1105,20111105,0.397133,0.285333,0.418938,0.324051,0.659893,0.308411,69.072000
4,20101106,0.396529,0.310861,0.439607,0.356234,0.685307,0.361353,66.285714,2010,1106,20111106,0.397741,0.292220,0.412838,0.360759,0.664492,0.361630,64.621875


In [18]:

corrs = []
cols = ['FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']

for col in cols:
    ncol = 'N'+col
    corrs.append(np.round(tgb[col].corr(tgb[ncol]),2))


pdf = pd.DataFrame([corrs],columns=cols)

In [19]:
# check how many out of top 5 contributors return
players = pd.read_csv(output+'PL_Returning.csv')

players.head()

,PlayerID,Season,TeamID,PlayerName,AltName,Returning,assist,block,cr_2pt,made1_free,made2_jump,made3_jump,rebound,steal,turnover
0,600001,2010,1102,BOHANNON_ZACH,ZACH_BOHANNON,True,7.0,0.0,10.0,4.0,2.0,2.0,17.0,7.0,6.0
1,600002,2010,1102,BRAKEVILLE_ADAM,ADAM_BRAKEVILLE,True,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,600003,2010,1102,BROEKHUIS_TAYLOR,TAYLOR_BROEKHUIS,True,11.0,9.0,14.0,25.0,6.0,1.0,54.0,4.0,22.0
3,600004,2010,1102,BROOKS_DEREK,DEREK_BROOKS,True,6.0,6.0,15.0,15.0,8.0,10.0,25.0,6.0,11.0
4,600005,2010,1102,EL-AMIN_SAJ,SAJ_EL-AMIN,False,1.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0


In [20]:
# calculate a players 'value'
players['PV'] = 1.25*(players['assist']+players['block']+players['rebound']) + 2 * (players['cr_2pt']+players['made2_jump'])+ \
players['made1_free']+3*players['made3_jump'] + 1.5*players['steal'] - 1.25 * players['turnover']

players.head()


,PlayerID,Season,TeamID,PlayerName,AltName,Returning,assist,block,cr_2pt,made1_free,made2_jump,made3_jump,rebound,steal,turnover,PV
0,600001,2010,1102,BOHANNON_ZACH,ZACH_BOHANNON,True,7.0,0.0,10.0,4.0,2.0,2.0,17.0,7.0,6.0,67.00
1,600002,2010,1102,BRAKEVILLE_ADAM,ADAM_BRAKEVILLE,True,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.25
2,600003,2010,1102,BROEKHUIS_TAYLOR,TAYLOR_BROEKHUIS,True,11.0,9.0,14.0,25.0,6.0,1.0,54.0,4.0,22.0,139.00
3,600004,2010,1102,BROOKS_DEREK,DEREK_BROOKS,True,6.0,6.0,15.0,15.0,8.0,10.0,25.0,6.0,11.0,132.50
4,600005,2010,1102,EL-AMIN_SAJ,SAJ_EL-AMIN,False,1.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,9.25


In [21]:
players=players[['PlayerID','Season','TeamID','Returning','PV']]
players['SeaTeamID'] = players['Season'].astype(str)+players['TeamID'].astype(str)



In [22]:
players['PV_Rank'] = players.groupby(['SeaTeamID'])['PV'].rank(ascending=False)
players = players.sort_values(by=['Season','TeamID','PV_Rank'],ascending=True)

players.head()

,PlayerID,Season,TeamID,Returning,PV,SeaTeamID,PV_Rank
16,600018,2010,1102,True,532.25,20101102,1.0
7,600008,2010,1102,True,351.75,20101102,2.0
10,600011,2010,1102,True,348.00,20101102,3.0
13,600014,2010,1102,False,341.25,20101102,4.0
11,600012,2010,1102,False,264.00,20101102,5.0


In [23]:
players = players.loc[players['PV_Rank']<=5]
players['Returning'] = players['Returning'].astype(int)

In [24]:
returning = players.groupby(['SeaTeamID'])['Returning'].sum().reset_index()
returning.head(10)

,SeaTeamID,Returning
0,20101102,3
1,20101103,3
2,20101104,3
3,20101105,4
4,20101106,3
5,20101107,3
6,20101108,2
7,20101110,5
8,20101111,3
9,20101112,4


In [25]:
tgb = pd.merge(tgb, returning, how='left', on=['SeaTeamID','SeaTeamID'])
tgb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace,Season,TeamID,NextSeaTID,NFG%,NFGM3%,NA_FG%,NA_FGM3%,NDRR,NORR,NPace,Returning
0,20101102,0.438729,0.310476,0.455344,0.346026,0.691477,0.233333,57.662069,2010,1102,20111102,0.468657,0.376258,0.411141,0.337681,0.656904,0.203980,61.575000,3.0
1,20101103,0.435331,0.338509,0.423662,0.325397,0.665187,0.376910,66.060606,2010,1103,20111103,0.429820,0.360215,0.417017,0.318339,0.669106,0.284526,66.502941,3.0
2,20101104,0.440914,0.350000,0.424452,0.317544,0.658363,0.349638,64.268750,2010,1104,20111104,0.444941,0.298405,0.383353,0.318267,0.663221,0.372458,64.521875,3.0
3,20101105,0.370342,0.282051,0.436211,0.313830,0.586085,0.339323,70.213043,2010,1105,20111105,0.397133,0.285333,0.418938,0.324051,0.659893,0.308411,69.072000,4.0
4,20101106,0.396529,0.310861,0.439607,0.356234,0.685307,0.361353,66.285714,2010,1106,20111106,0.397741,0.292220,0.412838,0.360759,0.664492,0.361630,64.621875,3.0


In [26]:
print(len(tgb))
tgb = tgb.dropna()
print(len(tgb))

3139
2785


In [27]:
sample_sizes = tgb.groupby(['Returning'])['SeaTeamID'].count().values
sample_sizes

array([ 60, 290, 753, 872, 629, 181])

In [28]:
# testing that random shuffles lead to no correlation
# test_col = tgb['A_FG%'].copy().values

# np.random.shuffle(test_col)

# tgb['A_FG%'] = test_col
# np.random.shuffle(test_col)
# tgb['FG%'] = test_col


In [29]:
cols = ['FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']

rows = []

for num_returning in range(6):
    sub_df = tgb.loc[tgb['Returning']==num_returning]
    corrs=[]
    for col in cols:
        ncol = 'N'+col
        corrs.append(np.round(sub_df[col].corr(sub_df[ncol]),2))
    rows.append(corrs)
    
# 'All' column
corrs = []
for col in cols:
    ncol = 'N'+col
    corrs.append(np.round(tgb[col].corr(tgb[ncol]),2))
rows.append(corrs)

pdf = pd.DataFrame(rows,columns=cols)


In [30]:
pdf['Num_Returning'] = [0,1,2,3,4,5,-1]

sample_sizes = np.append(sample_sizes,sample_sizes.sum())
pdf['Sample Size'] = sample_sizes
pdf = pdf[['Num_Returning','Sample Size','FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']]


pdf =pdf.sort_values(by=['Num_Returning'], ascending=False).reset_index(drop=True)
pdf.loc[:,'Num_Returning'] = pdf.Num_Returning.copy().replace(-1,'All')

pdf

,Num_Returning,Sample Size,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace
0,5,181,0.62,0.37,0.60,0.23,0.66,0.74,0.63
1,4,629,0.60,0.39,0.57,0.24,0.69,0.70,0.61
2,3,872,0.62,0.40,0.61,0.26,0.65,0.68,0.64
3,2,753,0.54,0.41,0.59,0.25,0.60,0.64,0.55
4,1,290,0.59,0.44,0.45,0.26,0.57,0.59,0.47
5,0,60,0.59,0.30,0.52,0.39,0.46,0.45,0.62
6,All,2785,0.57,0.39,0.57,0.25,0.63,0.66,0.59
